In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os, sys

parent_dir = os.path.split(os.getcwd())[0]
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

Целевая архитектура библиотеки - ЛД 3.0, поэтому используем Spark

Для отладки моделей (не на кластере) создаём локальную сессию

In [3]:
from sponge_bob_magic.session_handler import State

# объект State позволяет использовать одну и ту же сессию Spark в разных объектах
# если сессия уже инициализированна, то её нужно передать при инициализации Spate
spark = State().session
spark

## Подготовка данных <a name='data-preparator'></a>
Библиотека содержит утилиты для скачивания и парсинга популярных датасетов для рекомендаций

In [4]:
from sponge_bob_magic.datasets.movielens import MovieLens

data = MovieLens("1m")
data.ratings.to_csv("data.csv", index=False)
data.info()

ratings


,user_id,item_id,relevance,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968



users


,user_id,gender,age,occupation,zip_code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117



items


,item_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


На данный момент модуль загрузки открытых данных не использует Spark

В этой ячейке пример того, как можно использовать свои данные в формате CSV

In [5]:
from sponge_bob_magic.data_preparator.data_preparator import DataPreparator

log = DataPreparator(spark).transform_log(
    "data.csv",
    columns_names={
        "user_id": "user_id",
        "item_id": "item_id",
        "relevance": "relevance",
        "timestamp": "timestamp"
    },
    header=True,
    format_type="csv"
)

Библиотека содержит различные схемы валидации рекомендательных систем, встречающиеся в литературе

Пользователь может выбрать ту, которая лучше подходит бизнес-задаче

In [8]:
from sponge_bob_magic.splitters import UserSplitter

splitter = UserSplitter(
    drop_cold_items=True,
    drop_cold_users=True,
    item_test_size=1,
    user_test_size=500,
    seed=1234,
    shuffle=True
)
train, test_input, test = splitter._split_quantity(log)
(
    train.count(), 
    test_input.count(), 
    test.count()
)

(999703, 999703, 506)

## NMF
Простейший пример использования DL в рекомендациях

In [9]:
# простейший пример использования DL в рекомендациях
from sponge_bob_magic.models.neuromf_rec import NeuroMFRec

nmf = NeuroMFRec(
    learning_rate=0.01,
    epochs=10,
    embedding_dimension=100
)

In [10]:
%%time

nmf.fit(
    log=train,
    user_features=None,
    item_features=None
)

CPU times: user 14.7 s, sys: 2.4 s, total: 17.1 s
Wall time: 46 s


In [11]:
%%time

recs = nmf.predict(
    k=10,
    users=test.select('user_id').distinct(),
    items=test.select('item_id').distinct(),
    context='no_context',
    log=train,
    user_features=None,
    item_features=None,
    filter_seen_items=True
)

CPU times: user 1.36 s, sys: 104 ms, total: 1.46 s
Wall time: 13.6 s


В библиотеке реализовано несколько различных метрик качества рекомендательных систем, встречающихся в литературе

Пользователь может выбрать те, которые лучше подходят бизнес-задаче

In [12]:
from sponge_bob_magic.metrics import HitRate

hit_rate = HitRate()

hit_rate(recs, test, k=10)

0.03162055335968379

In [13]:
from sponge_bob_magic.metrics import NDCG

ndcg = NDCG()

ndcg(recs, test, k=10)

0.018711146130907613

## ALS
Библиотека также содержит классические алгоритмы рекомендаций, например, матричную факторизацию

In [14]:
from sponge_bob_magic.models.als_rec import ALSRec

als = ALSRec(rank=100)

In [15]:
%%time
als.fit(
    log=train,
    user_features=None,
    item_features=None
)

CPU times: user 24 ms, sys: 12 ms, total: 36 ms
Wall time: 32.7 s


In [16]:
%%time
recs = als.predict(
    k=10,
    users=test.select('user_id').distinct(),
    items=test.select('item_id').distinct(),
    context='no_context',
    log=train,
    user_features=None,
    item_features=None,
    filter_seen_items=True
)

CPU times: user 24 ms, sys: 8 ms, total: 32 ms
Wall time: 11.3 s


In [17]:
hit_rate(recs, test, k=10)

0.12055335968379446

In [18]:
ndcg(recs, test, k=10)

0.04913283620755997

## Своя модель
Для построения своей модели нужно использовать тот же самый split, что и для бейзлайнов

In [19]:
train.toPandas().to_csv("train.csv", index=False)

In [20]:
!head train.csv

user_id,item_id,relevance,timestamp,context
1090,2433,3.0,2000-11-23 01:26:01,no_context
1090,1247,4.0,2000-11-23 01:21:02,no_context
1090,954,4.0,2000-11-23 01:21:50,no_context
1090,3386,3.0,2000-11-23 01:14:07,no_context
1090,1939,3.0,2000-11-23 01:20:32,no_context
1090,2428,2.0,2000-11-23 01:27:11,no_context
1090,1673,3.0,2000-11-23 01:05:15,no_context
1090,1645,4.0,2000-11-23 01:33:11,no_context
1090,1093,3.0,2000-11-23 01:26:44,no_context


Здесь магия: можно взять train и обучить на нём свою любимую модель вне библиотеки

Также нужно выдать рекомендации обученной моделью

In [21]:
recs.toPandas().to_csv("recs.csv", index=False)

Теперь нужно прочитать рекомендации в формате, поддерживаемом библиотекой

In [22]:
recs = DataPreparator(spark).transform_log(
    "recs.csv",
    columns_names={
        "user_id": "user_id",
        "item_id": "item_id",
        "relevance": "relevance"
    },
    header=True,
    format_type="csv"
)

и сравнить качество своей модели с бейзлайнамии

In [23]:
# и сравнить качество своей модели с бейзлайнамии
hit_rate(recs, test, k=10)

0.12055335968379446